##### Sprintの目的
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

#### 公式Example

深層学習フレームワークには公式に様々なモデルのExampleコードが公開されています。

### 【問題1】公式チュートリアルモデルを分担して実行
TensorFLowの公式チュートリアルモデルを分担して実行してください。


以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。


[models/tutorials at master · tensorflow/models](https://www.tensorflow.org/tutorials/)



#### [Kerasチューナーの紹介](https://www.tensorflow.org/tutorials/keras/keras_tuner)

##### 概観
Keras Tunerは、TensorFlowプログラムに最適なハイパーパラメータのセットを選択するのに役立つライブラリです。機械学習（ML）アプリケーションに適したハイパーパラメーターのセットを選択するプロセスは、 ハイパーパラメーター チューニングまたはハイパーチューニングと呼ばれます。

ハイパーパラメータは、トレーニングプロセスとMLモデルのトポロジを管理する変数です。これらの変数はトレーニングプロセス全体で一定であり、MLプログラムのパフォーマンスに直接影響します。ハイパーパラメータには次の2つのタイプがあります。

- 非表示レイヤーの数や幅などのモデル選択に影響を与えるモデルハイパーパラメーター
- 確率的勾配降下法（SGD）の学習率やk最近傍（KNN）分類器の最近傍の数など、学習アルゴリズムの速度と品質に影響を与えるアルゴリズムハイパーパラメーター


このチュートリアルでは、Keras Tunerを使用して、画像分類アプリケーションのハイパーチューニングを実行します。

##### Setup

In [2]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [3]:
#!pip install -U keras-tuner
import kerastuner as kt

In [4]:
tf.__version__

'1.14.0'

In [5]:
tf.keras.__version__

'2.2.4-tf'

##### データセットの用意
このチュートリアルでは、Keras Tunerを使用して、 Fashion MNISTデータセットから衣服の画像を分類する機械学習モデルに最適なハイパーパラメーターを見つけます。

データをロードします。

In [6]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [7]:
# Normalize pixel values between 0 and 1
#正規化

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [8]:
display(img_train)
print(img_train.shape)
print("-------------------")
display(label_train)
print(label_train.shape)

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

(60000, 28, 28)
-------------------


array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

(60000,)


In [9]:
np.unique(label_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

ラベル数は10

##### モデルを定義する
ハイパーチューニング用のモデルを構築するときは、モデルアーキテクチャに加えて、ハイパーパラメーター検索スペースも定義します。あなたがhypertuning用に設定したモデルはhypermodelと呼ばれます。

次の2つの方法でハイパーモデルを定義できます。

- モデルビルダー機能を使用する
- HyperModel Tuner APIのHyperModelクラスをサブクラス化する

コンピュータビジョンアプリケーションには、 HyperXceptionとHyperResNetの 2つの定義済みHyperModelクラスを使用することもできます。

このチュートリアルでは、モデルビルダー関数を使用して画像分類モデルを定義します。モデルビルダー関数はコンパイルされたモデルを返し、インラインで定義したハイパーパラメーターを使用してモデルをハイパーチューニングします。

In [10]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32) #15パターン
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu')) #
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])  #3パターン

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

    return model

keras.losses.SparseCategoricalCrossentropy
>ラベルと予測の間のクロスエントロピー損失を計算します。

##### チューナーをインスタンス化してハイパーチューニングを実行する
チューナーをインスタンス化してハイパーチューニングを実行します。

Kerasチューナーには、RandomSearch、Hyperband、BayesianOptimization、およびSklearnの4つのチューナーがあります。 このチュートリアルでは、ハイパーバンドチューナーを使用します。

ハイパーバンドチューナーをインスタンス化するには、ハイパーモデル、最適化の目的 、トレーニングするエポックの最大数（ max_epochs ）を指定する必要があります。

In [11]:
tuner = kt.Hyperband(model_builder,
                     objective = "val_acc", 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


factorはモデルの減らし方をコントロール??（1/3ずつ）

>実行時、下記エラーが発生したので、tensorflow1.14.0にアップグレード
>
>AttributeError: module 'tensorflow._api.v1.keras.metrics' has no attribute 'Metric'.

>2つ下のCodeセルで下記エラー発生したので、'val_accuracy'→'val_acc'
>
>ValueError: Objective value missing in metrics reported to the Oracle, expected: ['val_accuracy'], found: dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

ハイパーバンドチューニングアルゴリズムは、適応型リソース割り当てと早期停止を使用して、高性能モデルにすばやく収束します。これは、スポーツ選手権スタイルのブラケットを使用して行われます。  
アルゴリズムは、少数のエポックで多数のモデルをトレーニングし、モデルの上位の半分のみを次のラウンドに繰り越します。   
ハイパーバンドは、1 + log`factor`（max_epochs）を計算し、最も近い整数に切り上げることにより、ブラケットでトレーニングするモデルの数を決定します。

ハイパーパラメーター検索を実行する前に、すべてのトレーニングステップの最後にトレーニング出力をクリアするコールバックを定義します。

In [12]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

ハイパーパラメータ検索を実行します。検索メソッドの引数は、上記のコールバックに加えて、 tf.keras.model.fitに使用されるものと同じです。


In [13]:
%%time

tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.

CPU times: user 22min 16s, sys: 1min 15s, total: 23min 31s
Wall time: 4min 42s


In [14]:
tuner.search_space_summary()

In [15]:
tuner.results_summary()

このチュートリアルを完了するには、検索からの最適なハイパーパラメーターを使用してモデルを再トレーニングします。

In [16]:
# Build the model with the optimal hyperparameters and train it on the data
#最適なハイパーパラメーターを使用してモデルを作成し、データでトレーニングする

model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 4:27 - loss: 2.5490 - acc: 0.031 - ETA: 24s - loss: 1.5289 - acc: 0.491 - ETA: 15s - loss: 1.2614 - acc: 0.56 - ETA: 12s - loss: 1.0961 - acc: 0.61 - ETA: 11s - loss: 1.0155 - acc: 0.64 - ETA: 10s - loss: 0.9447 - acc: 0.67 - ETA: 9s - loss: 0.8793 - acc: 0.6922 - ETA: 9s - loss: 0.8448 - acc: 0.705 - ETA: 8s - loss: 0.8222 - acc: 0.714 - ETA: 8s - loss: 0.7991 - acc: 0.723 - ETA: 8s - loss: 0.7837 - acc: 0.728 - ETA: 7s - loss: 0.7614 - acc: 0.736 - ETA: 7s - loss: 0.7430 - acc: 0.742 - ETA: 7s - loss: 0.7287 - acc: 0.746 - ETA: 7s - loss: 0.7194 - acc: 0.748 - ETA: 6s - loss: 0.7111 - acc: 0.750 - ETA: 6s - loss: 0.7031 - acc: 0.752 - ETA: 6s - loss: 0.6994 - acc: 0.752 - ETA: 6s - loss: 0.7002 - acc: 0.751 - ETA: 6s - loss: 0.6924 - acc: 0.754 - ETA: 6s - loss: 0.6847 - acc: 0.757 - ETA: 6s - loss: 0.6781 - acc: 0.759 - ETA: 6s - loss: 0.6685 - acc: 0.763 - ETA: 6s - loss

60000/60000 [==============================] - ETA: 10s - loss: 0.2945 - acc: 0.87 - ETA: 6s - loss: 0.3902 - acc: 0.8672 - ETA: 6s - loss: 0.3933 - acc: 0.861 - ETA: 6s - loss: 0.3766 - acc: 0.861 - ETA: 6s - loss: 0.3809 - acc: 0.862 - ETA: 6s - loss: 0.3783 - acc: 0.859 - ETA: 6s - loss: 0.3714 - acc: 0.860 - ETA: 6s - loss: 0.3760 - acc: 0.857 - ETA: 6s - loss: 0.3811 - acc: 0.855 - ETA: 6s - loss: 0.3849 - acc: 0.855 - ETA: 6s - loss: 0.3852 - acc: 0.856 - ETA: 5s - loss: 0.3872 - acc: 0.856 - ETA: 5s - loss: 0.3902 - acc: 0.856 - ETA: 5s - loss: 0.3893 - acc: 0.856 - ETA: 5s - loss: 0.3842 - acc: 0.858 - ETA: 5s - loss: 0.3834 - acc: 0.858 - ETA: 5s - loss: 0.3822 - acc: 0.858 - ETA: 5s - loss: 0.3818 - acc: 0.857 - ETA: 5s - loss: 0.3833 - acc: 0.857 - ETA: 5s - loss: 0.3806 - acc: 0.857 - ETA: 5s - loss: 0.3781 - acc: 0.859 - ETA: 5s - loss: 0.3752 - acc: 0.860 - ETA: 5s - loss: 0.3708 - acc: 0.861 - ETA: 5s - loss: 0.3682 - acc: 0.862 - ETA: 5s - loss: 0.3690 - acc: 0.862 - ET

60000/60000 [==============================] - ETA: 9s - loss: 0.4150 - acc: 0.843 - ETA: 6s - loss: 0.3537 - acc: 0.873 - ETA: 6s - loss: 0.3424 - acc: 0.870 - ETA: 6s - loss: 0.3538 - acc: 0.872 - ETA: 5s - loss: 0.3344 - acc: 0.877 - ETA: 5s - loss: 0.3452 - acc: 0.872 - ETA: 5s - loss: 0.3359 - acc: 0.873 - ETA: 5s - loss: 0.3279 - acc: 0.876 - ETA: 5s - loss: 0.3236 - acc: 0.878 - ETA: 5s - loss: 0.3151 - acc: 0.882 - ETA: 5s - loss: 0.3112 - acc: 0.884 - ETA: 5s - loss: 0.3106 - acc: 0.884 - ETA: 5s - loss: 0.3071 - acc: 0.885 - ETA: 5s - loss: 0.3063 - acc: 0.886 - ETA: 5s - loss: 0.3085 - acc: 0.885 - ETA: 5s - loss: 0.3062 - acc: 0.887 - ETA: 5s - loss: 0.3070 - acc: 0.887 - ETA: 5s - loss: 0.3093 - acc: 0.887 - ETA: 4s - loss: 0.3099 - acc: 0.887 - ETA: 4s - loss: 0.3078 - acc: 0.888 - ETA: 4s - loss: 0.3057 - acc: 0.888 - ETA: 4s - loss: 0.3087 - acc: 0.887 - ETA: 4s - loss: 0.3106 - acc: 0.887 - ETA: 4s - loss: 0.3127 - acc: 0.886 - ETA: 4s - loss: 0.3129 - acc: 0.886 - ETA

60000/60000 [==============================] - ETA: 9s - loss: 0.2079 - acc: 0.906 - ETA: 7s - loss: 0.2844 - acc: 0.891 - ETA: 6s - loss: 0.2913 - acc: 0.893 - ETA: 6s - loss: 0.2931 - acc: 0.895 - ETA: 6s - loss: 0.2893 - acc: 0.896 - ETA: 6s - loss: 0.2818 - acc: 0.898 - ETA: 6s - loss: 0.2804 - acc: 0.896 - ETA: 5s - loss: 0.2838 - acc: 0.896 - ETA: 5s - loss: 0.2872 - acc: 0.894 - ETA: 5s - loss: 0.2878 - acc: 0.894 - ETA: 5s - loss: 0.2888 - acc: 0.894 - ETA: 5s - loss: 0.2900 - acc: 0.894 - ETA: 5s - loss: 0.2869 - acc: 0.894 - ETA: 5s - loss: 0.2879 - acc: 0.894 - ETA: 5s - loss: 0.2864 - acc: 0.895 - ETA: 5s - loss: 0.2848 - acc: 0.895 - ETA: 5s - loss: 0.2826 - acc: 0.897 - ETA: 5s - loss: 0.2840 - acc: 0.896 - ETA: 5s - loss: 0.2859 - acc: 0.895 - ETA: 4s - loss: 0.2877 - acc: 0.895 - ETA: 4s - loss: 0.2866 - acc: 0.895 - ETA: 4s - loss: 0.2886 - acc: 0.895 - ETA: 4s - loss: 0.2893 - acc: 0.895 - ETA: 4s - loss: 0.2901 - acc: 0.895 - ETA: 4s - loss: 0.2908 - acc: 0.895 - ETA

60000/60000 [==============================] - ETA: 9s - loss: 0.3463 - acc: 0.906 - ETA: 6s - loss: 0.2709 - acc: 0.906 - ETA: 5s - loss: 0.2792 - acc: 0.899 - ETA: 5s - loss: 0.2749 - acc: 0.898 - ETA: 5s - loss: 0.2813 - acc: 0.896 - ETA: 5s - loss: 0.2879 - acc: 0.893 - ETA: 5s - loss: 0.2838 - acc: 0.895 - ETA: 5s - loss: 0.2819 - acc: 0.895 - ETA: 5s - loss: 0.2818 - acc: 0.896 - ETA: 5s - loss: 0.2826 - acc: 0.896 - ETA: 5s - loss: 0.2861 - acc: 0.895 - ETA: 5s - loss: 0.2866 - acc: 0.895 - ETA: 5s - loss: 0.2815 - acc: 0.896 - ETA: 5s - loss: 0.2807 - acc: 0.897 - ETA: 5s - loss: 0.2817 - acc: 0.896 - ETA: 5s - loss: 0.2818 - acc: 0.896 - ETA: 5s - loss: 0.2803 - acc: 0.896 - ETA: 4s - loss: 0.2806 - acc: 0.896 - ETA: 4s - loss: 0.2781 - acc: 0.896 - ETA: 4s - loss: 0.2808 - acc: 0.896 - ETA: 4s - loss: 0.2809 - acc: 0.896 - ETA: 4s - loss: 0.2810 - acc: 0.896 - ETA: 4s - loss: 0.2822 - acc: 0.896 - ETA: 4s - loss: 0.2852 - acc: 0.894 - ETA: 4s - loss: 0.2859 - acc: 0.894 - ETA

60000/60000 [==============================] - ETA: 10s - loss: 0.3898 - acc: 0.84 - ETA: 6s - loss: 0.3202 - acc: 0.8750 - ETA: 6s - loss: 0.2648 - acc: 0.904 - ETA: 6s - loss: 0.2642 - acc: 0.906 - ETA: 5s - loss: 0.2662 - acc: 0.905 - ETA: 5s - loss: 0.2739 - acc: 0.904 - ETA: 5s - loss: 0.2785 - acc: 0.902 - ETA: 5s - loss: 0.2762 - acc: 0.903 - ETA: 5s - loss: 0.2699 - acc: 0.905 - ETA: 5s - loss: 0.2691 - acc: 0.903 - ETA: 5s - loss: 0.2662 - acc: 0.903 - ETA: 5s - loss: 0.2593 - acc: 0.905 - ETA: 5s - loss: 0.2616 - acc: 0.904 - ETA: 5s - loss: 0.2600 - acc: 0.904 - ETA: 5s - loss: 0.2569 - acc: 0.905 - ETA: 5s - loss: 0.2577 - acc: 0.905 - ETA: 5s - loss: 0.2596 - acc: 0.905 - ETA: 5s - loss: 0.2578 - acc: 0.905 - ETA: 5s - loss: 0.2592 - acc: 0.905 - ETA: 5s - loss: 0.2607 - acc: 0.905 - ETA: 5s - loss: 0.2623 - acc: 0.904 - ETA: 4s - loss: 0.2603 - acc: 0.905 - ETA: 4s - loss: 0.2618 - acc: 0.905 - ETA: 4s - loss: 0.2594 - acc: 0.906 - ETA: 4s - loss: 0.2587 - acc: 0.906 - ET

60000/60000 [==============================] - ETA: 10s - loss: 0.3278 - acc: 0.87 - ETA: 5s - loss: 0.2350 - acc: 0.9132 - ETA: 5s - loss: 0.2303 - acc: 0.914 - ETA: 5s - loss: 0.2672 - acc: 0.897 - ETA: 5s - loss: 0.2621 - acc: 0.900 - ETA: 5s - loss: 0.2585 - acc: 0.904 - ETA: 5s - loss: 0.2565 - acc: 0.905 - ETA: 5s - loss: 0.2587 - acc: 0.906 - ETA: 5s - loss: 0.2590 - acc: 0.904 - ETA: 5s - loss: 0.2621 - acc: 0.902 - ETA: 5s - loss: 0.2650 - acc: 0.902 - ETA: 5s - loss: 0.2611 - acc: 0.903 - ETA: 5s - loss: 0.2583 - acc: 0.904 - ETA: 5s - loss: 0.2577 - acc: 0.905 - ETA: 5s - loss: 0.2575 - acc: 0.906 - ETA: 5s - loss: 0.2587 - acc: 0.905 - ETA: 4s - loss: 0.2615 - acc: 0.904 - ETA: 4s - loss: 0.2625 - acc: 0.903 - ETA: 4s - loss: 0.2624 - acc: 0.904 - ETA: 4s - loss: 0.2627 - acc: 0.904 - ETA: 4s - loss: 0.2631 - acc: 0.903 - ETA: 4s - loss: 0.2623 - acc: 0.904 - ETA: 4s - loss: 0.2620 - acc: 0.903 - ETA: 4s - loss: 0.2622 - acc: 0.903 - ETA: 4s - loss: 0.2619 - acc: 0.903 - ET

60000/60000 [==============================] - ETA: 9s - loss: 0.2644 - acc: 0.843 - ETA: 6s - loss: 0.2447 - acc: 0.898 - ETA: 6s - loss: 0.2329 - acc: 0.911 - ETA: 6s - loss: 0.2336 - acc: 0.908 - ETA: 5s - loss: 0.2334 - acc: 0.912 - ETA: 5s - loss: 0.2393 - acc: 0.909 - ETA: 5s - loss: 0.2434 - acc: 0.910 - ETA: 5s - loss: 0.2437 - acc: 0.909 - ETA: 5s - loss: 0.2422 - acc: 0.909 - ETA: 5s - loss: 0.2409 - acc: 0.909 - ETA: 5s - loss: 0.2407 - acc: 0.910 - ETA: 5s - loss: 0.2356 - acc: 0.911 - ETA: 5s - loss: 0.2385 - acc: 0.911 - ETA: 5s - loss: 0.2370 - acc: 0.911 - ETA: 5s - loss: 0.2378 - acc: 0.911 - ETA: 5s - loss: 0.2391 - acc: 0.910 - ETA: 5s - loss: 0.2393 - acc: 0.911 - ETA: 5s - loss: 0.2410 - acc: 0.910 - ETA: 5s - loss: 0.2377 - acc: 0.911 - ETA: 5s - loss: 0.2347 - acc: 0.913 - ETA: 4s - loss: 0.2341 - acc: 0.913 - ETA: 4s - loss: 0.2331 - acc: 0.913 - ETA: 4s - loss: 0.2314 - acc: 0.914 - ETA: 4s - loss: 0.2324 - acc: 0.913 - ETA: 4s - loss: 0.2328 - acc: 0.912 - ETA

60000/60000 [==============================] - ETA: 10s - loss: 0.3410 - acc: 0.90 - ETA: 6s - loss: 0.2773 - acc: 0.8926 - ETA: 6s - loss: 0.2567 - acc: 0.907 - ETA: 6s - loss: 0.2415 - acc: 0.912 - ETA: 6s - loss: 0.2368 - acc: 0.911 - ETA: 5s - loss: 0.2353 - acc: 0.912 - ETA: 5s - loss: 0.2301 - acc: 0.913 - ETA: 5s - loss: 0.2309 - acc: 0.913 - ETA: 5s - loss: 0.2276 - acc: 0.916 - ETA: 5s - loss: 0.2253 - acc: 0.916 - ETA: 5s - loss: 0.2262 - acc: 0.915 - ETA: 5s - loss: 0.2309 - acc: 0.913 - ETA: 5s - loss: 0.2306 - acc: 0.914 - ETA: 5s - loss: 0.2277 - acc: 0.915 - ETA: 5s - loss: 0.2264 - acc: 0.916 - ETA: 5s - loss: 0.2264 - acc: 0.916 - ETA: 5s - loss: 0.2265 - acc: 0.915 - ETA: 5s - loss: 0.2258 - acc: 0.915 - ETA: 5s - loss: 0.2247 - acc: 0.916 - ETA: 4s - loss: 0.2249 - acc: 0.917 - ETA: 4s - loss: 0.2263 - acc: 0.915 - ETA: 4s - loss: 0.2263 - acc: 0.916 - ETA: 4s - loss: 0.2277 - acc: 0.915 - ETA: 4s - loss: 0.2295 - acc: 0.914 - ETA: 4s - loss: 0.2304 - acc: 0.914 - ET

60000/60000 [==============================] - ETA: 10s - loss: 0.3323 - acc: 0.90 - ETA: 7s - loss: 0.2034 - acc: 0.9250 - ETA: 7s - loss: 0.1936 - acc: 0.927 - ETA: 6s - loss: 0.1907 - acc: 0.927 - ETA: 6s - loss: 0.2029 - acc: 0.925 - ETA: 6s - loss: 0.2028 - acc: 0.925 - ETA: 5s - loss: 0.2084 - acc: 0.921 - ETA: 5s - loss: 0.2065 - acc: 0.923 - ETA: 5s - loss: 0.2032 - acc: 0.926 - ETA: 5s - loss: 0.2047 - acc: 0.924 - ETA: 5s - loss: 0.2032 - acc: 0.925 - ETA: 5s - loss: 0.2034 - acc: 0.925 - ETA: 5s - loss: 0.2070 - acc: 0.923 - ETA: 5s - loss: 0.2085 - acc: 0.922 - ETA: 5s - loss: 0.2104 - acc: 0.921 - ETA: 5s - loss: 0.2109 - acc: 0.921 - ETA: 5s - loss: 0.2115 - acc: 0.920 - ETA: 5s - loss: 0.2117 - acc: 0.920 - ETA: 5s - loss: 0.2140 - acc: 0.919 - ETA: 5s - loss: 0.2138 - acc: 0.919 - ETA: 4s - loss: 0.2139 - acc: 0.919 - ETA: 4s - loss: 0.2140 - acc: 0.919 - ETA: 4s - loss: 0.2156 - acc: 0.918 - ETA: 4s - loss: 0.2152 - acc: 0.917 - ETA: 4s - loss: 0.2162 - acc: 0.917 - ET

my_dir/intro_to_ktディレクトリには、 my_dir/intro_to_kt検索中に実行されたすべてのトライアル（モデル構成）の詳細なログとチェックポイントが含まれています。ハイパーパラメータ検索を再実行すると、Keras Tunerはこれらのログの既存の状態を使用して検索を再開します。この動作を無効にするには、チューナーをインスタンス化するときに追加のoverwrite = True引数を渡します。

In [17]:
tuner = kt.Hyperband(model_builder,
                     objective = "val_acc", 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [18]:
tuner.results_summary()

このチュートリアルでは、Keras Tunerを使用してモデルのハイパーパラメーターを調整する方法を学びました。 Keras Tunerの詳細については、次の追加リソースをご覧ください。

- [TensorFlowブログのKeras Tuner](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [Keras TunerのWebサイト](https://keras-team.github.io/keras-tuner/)

また、 TensorBoardのHParams Dashboardをチェックして、モデルのハイパーパラメーターをインタラクティブに調整します。

----

#### 異なるフレームワークへの書き換え

「ディープラーニングフレームワーク1」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。


- Iris（Iris-versicolorとIris-virginicaのみの2値分類）
- Iris（3種類全ての目的変数を使用して多値分類）
- House Prices
- MNIST

##### Kerasへの書き換え
KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。


KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。

### 【問題3】Iris（2値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。



In [51]:
df = pd.read_csv("Iris.csv")
display(df.head())
df.shape

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


(150, 6)

In [20]:
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

#データ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [21]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import CSVLogger

#学習
model = Sequential()
model.add(Dense(input_dim=4, output_dim=50, bias=True, activation='relu'))
model.add(Dense(input_dim=50, output_dim=100, bias=True, activation='relu'))
model.add(Dense(input_dim=100, output_dim=1, bias=True, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
csv_logger = CSVLogger('log.csv', separator=',', append=False)
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=100,
                    validation_data=(X_test, y_test)) #callbacks=[csv_logger]を追加でlogをcsvで保存可能

Using TensorFlow backend.
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=4, activation="relu", units=50, use_bias=True)`
  if __name__ == '__main__':
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, activation="relu", units=100, use_bias=True)`
  # Remove the CWD from sys.path while we load stuff.
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, activation="sigmoid", units=1, use_bias=True)`
  # This is added back by InteractiveShellApp.init_path()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 80 samples, validate on 20 samples
Epoch 1/100
80/80 [==============================] - ETA: 0s - loss: 0.7827 - accuracy: 0.43 - 1s 7ms/step - loss: 0.7311 - accuracy: 0.5375 - val_loss: 0.7067 - val_accuracy: 0.3500
Epoch 2/100
80/80 [==============================] - ETA: 0s - loss: 0.6520 - accuracy: 0.59 - 0s 125us/step - loss: 0.6740 - accuracy: 0.5375 - val_loss: 0.8020 - val_accuracy: 0.3500
Epoch 3/100
80/80 [==============================] - ETA: 0s - loss: 0.6963 - accuracy: 0.50 - 0s 124us/step - loss: 0.6695 - accuracy: 0.5375 - val_loss: 0.7774 - val_accuracy: 0.3500
Epoch 4/100
80/80 [==============================] - ETA: 0s - loss: 0.6662 - accuracy: 0.53 - 0s 127us/step - loss: 0.6539 - accuracy: 0.5375 - val_loss: 0.7153 - val_accuracy: 0.3500
Epoch 5/100
80/80 [==============================] - ETA: 0s - loss: 0.6552 - accuracy: 0.50 - 0s 136us/step - loss: 0.6387

Epoch 42/100
80/80 [==============================] - ETA: 0s - loss: 0.3252 - accuracy: 0.90 - 0s 135us/step - loss: 0.2941 - accuracy: 0.9250 - val_loss: 0.3269 - val_accuracy: 1.0000
Epoch 43/100
80/80 [==============================] - ETA: 0s - loss: 0.3243 - accuracy: 0.87 - 0s 123us/step - loss: 0.2872 - accuracy: 0.9500 - val_loss: 0.3035 - val_accuracy: 1.0000
Epoch 44/100
80/80 [==============================] - ETA: 0s - loss: 0.2711 - accuracy: 0.96 - 0s 131us/step - loss: 0.2791 - accuracy: 0.9625 - val_loss: 0.2983 - val_accuracy: 1.0000
Epoch 45/100
80/80 [==============================] - ETA: 0s - loss: 0.3069 - accuracy: 0.96 - 0s 126us/step - loss: 0.2696 - accuracy: 0.9625 - val_loss: 0.3024 - val_accuracy: 1.0000
Epoch 46/100
80/80 [==============================] - ETA: 0s - loss: 0.2652 - accuracy: 0.96 - 0s 116us/step - loss: 0.2667 - accuracy: 0.9500 - val_loss: 0.3046 - val_accuracy: 1.0000
Epoch 47/100
80/80 [==============================] - ETA: 0s - loss: 

Epoch 86/100
80/80 [==============================] - ETA: 0s - loss: 0.1441 - accuracy: 0.96 - 0s 118us/step - loss: 0.1408 - accuracy: 0.9625 - val_loss: 0.1601 - val_accuracy: 1.0000
Epoch 87/100
80/80 [==============================] - ETA: 0s - loss: 0.0986 - accuracy: 1.00 - 0s 121us/step - loss: 0.1403 - accuracy: 0.9750 - val_loss: 0.1518 - val_accuracy: 1.0000
Epoch 88/100
80/80 [==============================] - ETA: 0s - loss: 0.1546 - accuracy: 0.96 - 0s 116us/step - loss: 0.1354 - accuracy: 0.9750 - val_loss: 0.1500 - val_accuracy: 1.0000
Epoch 89/100
80/80 [==============================] - ETA: 0s - loss: 0.1082 - accuracy: 1.00 - 0s 123us/step - loss: 0.1367 - accuracy: 0.9625 - val_loss: 0.1490 - val_accuracy: 1.0000
Epoch 90/100
80/80 [==============================] - ETA: 0s - loss: 0.1143 - accuracy: 0.96 - 0s 115us/step - loss: 0.1362 - accuracy: 0.9500 - val_loss: 0.1477 - val_accuracy: 1.0000
Epoch 91/100
80/80 [==============================] - ETA: 0s - loss: 

In [22]:
#推定
pred_train = np.round(model.predict(X_train)) #0.5以上なら1に切り上げ、未満なら0に切り下げ
pred_test = np.round(model.predict(X_test))

print(pred_train.reshape(1, -1))
print((pred_train == y_train).reshape(1, -1))
print("---------------------")
print(pred_test.reshape(1, -1))
print((pred_test == y_test).reshape(1, -1))

[[0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1.
  1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0.
  0. 1. 1. 0. 1. 1. 0. 0.]]
[[ True  True  True  True  True  True  True  True  True  True  True  True
  False  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True False  True  True  True
   True  True  True  True False  True  True  True  True  True  True  True
   True  True  True  True False  True  True  True]]
---------------------
[[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1.]]
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True]]


### 【問題4】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

#### 単純パーセプトロン

[入門 Keras (4) 多クラス分類 – Iris データを学習する](https://www.infiniteloop.co.jp/blog/2018/01/learning-keras-04/)

In [52]:
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

#kerasのto_categorical関数でone-hot-encodeing
from keras.utils import np_utils
y = np_utils.to_categorical(y)

#データ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#学習
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(input_dim=4, units=3)) # 入力が4種の特徴、3つに分離
model.add(Activation('softmax'))
#コンパイル時に損失関数と最適化手法、評価関数を指定
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=10) # エポック50, ミニバッチ 10

Epoch 1/50
120/120 [==============================] - ETA: 1s - loss: 5.3658 - accuracy: 0.30 - 0s 1ms/step - loss: 1.8835 - accuracy: 0.3833
Epoch 2/50
120/120 [==============================] - ETA: 0s - loss: 0.4185 - accuracy: 1.00 - 0s 87us/step - loss: 1.3109 - accuracy: 0.6000
Epoch 3/50
120/120 [==============================] - ETA: 0s - loss: 0.7759 - accuracy: 0.60 - 0s 114us/step - loss: 0.7584 - accuracy: 0.6750
Epoch 4/50
120/120 [==============================] - ETA: 0s - loss: 1.0770 - accuracy: 0.50 - 0s 114us/step - loss: 0.6905 - accuracy: 0.6667
Epoch 5/50
120/120 [==============================] - ETA: 0s - loss: 0.4933 - accuracy: 0.90 - 0s 112us/step - loss: 0.7553 - accuracy: 0.7000
Epoch 6/50
120/120 [==============================] - ETA: 0s - loss: 0.8076 - accuracy: 0.50 - 0s 132us/step - loss: 0.7250 - accuracy: 0.6917
Epoch 7/50
120/120 [==============================] - ETA: 0s - loss: 1.4014 - accuracy: 0.50 - 0s 145us/step - loss: 0.8624 - accuracy: 0.

In [53]:
#推定
y_pred = model.predict_classes(X_test, batch_size=10) # ミニバッチは訓練時と同じく 10

#評価
_, y_test_label = np.where(y_test > 0) #onr-hot表現をラベルに
print(y_pred == y_test_label)
print("----------------------------")

train_score = model.evaluate(X_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])
print("----------------------------")

test_score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

[ True  True  True  True  True  True  True False False  True  True  True
 False False False  True False False  True  True  True False  True  True
  True  True  True  True  True  True]
----------------------------
Train loss: 0.3027863085269928
Train accuracy: 0.8500000238418579
----------------------------
Test loss: 0.40847671031951904
Test accuracy: 0.7333333492279053


#### 3層ニューラルネットワーク

[KerasでIrisのデータセットを分類し、モデルを保存する](https://snova301.hatenablog.com/entry/2018/10/29/175028)

In [54]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import CSVLogger

#学習
model = Sequential()
model.add(Dense(input_dim=4, output_dim=50, bias=True, activation='relu'))
model.add(Dense(input_dim=50, output_dim=100, bias=True, activation='relu'))
model.add(Dense(input_dim=100, output_dim=3, bias=True, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
csv_logger = CSVLogger('log.csv', separator=',', append=False)
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=100,
                    validation_data=(X_test, y_test)) #callbacks=[csv_logger]を追加でlogをcsvで保存可能

/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=4, activation="relu", units=50, use_bias=True)`
  if __name__ == '__main__':
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, activation="relu", units=100, use_bias=True)`
  # Remove the CWD from sys.path while we load stuff.
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, activation="softmax", units=3, use_bias=True)`
  # This is added back by InteractiveShellApp.init_path()


Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - ETA: 1s - loss: 1.4394 - accuracy: 0.18 - 1s 4ms/step - loss: 1.2212 - accuracy: 0.3167 - val_loss: 1.0925 - val_accuracy: 0.2000
Epoch 2/100
120/120 [==============================] - ETA: 0s - loss: 1.0436 - accuracy: 0.40 - 0s 88us/step - loss: 1.0075 - accuracy: 0.3833 - val_loss: 1.0799 - val_accuracy: 0.2000
Epoch 3/100
120/120 [==============================] - ETA: 0s - loss: 1.0921 - accuracy: 0.15 - 0s 102us/step - loss: 0.9332 - accuracy: 0.5583 - val_loss: 1.0278 - val_accuracy: 0.5667
Epoch 4/100
120/120 [==============================] - ETA: 0s - loss: 0.9017 - accuracy: 0.68 - 0s 108us/step - loss: 0.8715 - accuracy: 0.6917 - val_loss: 0.9386 - val_accuracy: 0.5667
Epoch 5/100
120/120 [==============================] - ETA: 0s - loss: 0.7387 - accuracy: 0.84 - 0s 110us/step - loss: 0.8017 - accuracy: 0.6917 - val_loss: 0.8410 - val_accuracy: 0.5667
Epoch 6/100
120/120 [==

120/120 [==============================] - ETA: 0s - loss: 0.1704 - accuracy: 0.96 - 0s 109us/step - loss: 0.1609 - accuracy: 0.9667 - val_loss: 0.1277 - val_accuracy: 1.0000
Epoch 45/100
120/120 [==============================] - ETA: 0s - loss: 0.2393 - accuracy: 0.93 - 0s 105us/step - loss: 0.1545 - accuracy: 0.9750 - val_loss: 0.1240 - val_accuracy: 1.0000
Epoch 46/100
120/120 [==============================] - ETA: 0s - loss: 0.1374 - accuracy: 0.96 - 0s 109us/step - loss: 0.1500 - accuracy: 0.9750 - val_loss: 0.1256 - val_accuracy: 1.0000
Epoch 47/100
120/120 [==============================] - ETA: 0s - loss: 0.1182 - accuracy: 0.96 - 0s 109us/step - loss: 0.1484 - accuracy: 0.9667 - val_loss: 0.1195 - val_accuracy: 1.0000
Epoch 48/100
120/120 [==============================] - ETA: 0s - loss: 0.1667 - accuracy: 0.96 - 0s 98us/step - loss: 0.1436 - accuracy: 0.9750 - val_loss: 0.1177 - val_accuracy: 1.0000
Epoch 49/100
120/120 [==============================] - ETA: 0s - loss: 0.

Epoch 88/100
120/120 [==============================] - ETA: 0s - loss: 0.1751 - accuracy: 0.90 - 0s 117us/step - loss: 0.0901 - accuracy: 0.9667 - val_loss: 0.0437 - val_accuracy: 1.0000
Epoch 89/100
120/120 [==============================] - ETA: 0s - loss: 0.0978 - accuracy: 1.00 - 0s 111us/step - loss: 0.0899 - accuracy: 0.9750 - val_loss: 0.0452 - val_accuracy: 1.0000
Epoch 90/100
120/120 [==============================] - ETA: 0s - loss: 0.0749 - accuracy: 0.96 - 0s 100us/step - loss: 0.0917 - accuracy: 0.9667 - val_loss: 0.0410 - val_accuracy: 1.0000
Epoch 91/100
120/120 [==============================] - ETA: 0s - loss: 0.0976 - accuracy: 0.93 - 0s 102us/step - loss: 0.0872 - accuracy: 0.9667 - val_loss: 0.0423 - val_accuracy: 1.0000
Epoch 92/100
120/120 [==============================] - ETA: 0s - loss: 0.0544 - accuracy: 1.00 - 0s 97us/step - loss: 0.0913 - accuracy: 0.9667 - val_loss: 0.0482 - val_accuracy: 1.0000
Epoch 93/100
120/120 [==============================] - ETA: 

In [55]:
#推定
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)
pred_train = np.argmax(pred_train, axis=1) #一番大きな値のインデックスを出力
pred_test = np.argmax(pred_test, axis=1)


print(pred_train)
print(pred_train == np.argmax(y_train, axis=1))
print("---------------------")
print(pred_test)
print(pred_test == np.argmax(y_test, axis=1))

[2 1 0 2 2 1 0 2 1 1 2 0 2 0 0 1 2 2 2 2 1 2 1 1 2 2 2 2 1 2 1 0 2 1 1 1 1
 2 0 0 2 1 0 0 1 0 2 1 0 2 2 1 0 2 2 2 2 0 0 2 2 0 2 0 2 2 0 0 2 0 0 0 1 2
 2 0 0 0 1 1 0 0 1 0 2 1 2 1 0 2 0 2 0 0 2 0 2 1 1 1 2 2 2 1 0 1 2 2 0 1 1
 2 1 0 0 0 2 1 2 0]
[ True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True]
---------------------
[2

In [63]:
#評価
train_score = model.evaluate(X_train, y_train)
test_score = model.evaluate(X_test, y_test)

30/30 [==============================] - 0s 35us/step


In [64]:
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])
print("----------------------------")
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

Train loss: 0.08267993330955506
Train accuracy: 0.9666666388511658
----------------------------
Test loss: 0.03807102516293526
Test accuracy: 1.0


### 【問題5】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [5]:
# データセットの読み込み
df = pd.read_csv("train.csv")

# データフレームから条件抽出
X = df.loc[:, ["GrLivArea", "YearBuilt"]].values
y = df.loc[:, "SalePrice"].values

#標準化、対数変換
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
y_scaled = np.log(y)
y_scaled = y_scaled[:, np.newaxis]

#分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.callbacks import CSVLogger

#学習
model = Sequential()
model.add(Dense(input_dim=2, output_dim=50, bias=True, activation='relu'))
model.add(Dense(input_dim=50, output_dim=100, bias=True, activation='relu'))
model.add(Dense(input_dim=100, output_dim=1, bias=True))
model.compile(loss='mse', optimizer=Adam(), metrics=['mse'])
csv_logger = CSVLogger('log.csv', separator=',', append=False)
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=100,
                    validation_data=(X_test, y_test)) 

/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=2, activation="relu", units=50, use_bias=True)`
  if __name__ == '__main__':
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, activation="relu", units=100, use_bias=True)`
  # Remove the CWD from sys.path while we load stuff.
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1, use_bias=True)`
  # This is added back by InteractiveShellApp.init_path()



Train on 934 samples, validate on 292 samples
Epoch 1/100
934/934 [==============================] - 0s 511us/step - loss: 131.6629 - mse: 131.6629 - val_loss: 111.6445 - val_mse: 111.6445
Epoch 2/100
934/934 [==============================] - 0s 51us/step - loss: 84.8886 - mse: 84.8886 - val_loss: 48.4983 - val_mse: 48.4983
Epoch 3/100
934/934 [==============================] - 0s 60us/step - loss: 25.5102 - mse: 25.5102 - val_loss: 12.0500 - val_mse: 12.0500
Epoch 4/100
934/934 [==============================] - 0s 70us/step - loss: 9.7885 - mse: 9.7885 - val_loss: 10.8809 - val_mse: 10.8809
Epoch 5/100
934/934 [==============================] - 0s 62us/step - loss: 8.4111 - mse: 8.4111 - val_loss: 9.2961 - val_mse: 9.2961
Epoch 6/100
934/934 [==============================] - 0s 61us/step - loss: 7.1222 - mse: 7.1222 - val_loss: 7.8889 - val_mse: 7.8889
Epoch 7/100
934/934 [==============================] - 0s 61us/step - loss: 5.9819 - mse: 5.9819 - val_loss: 6.5693 - val_mse: 6.5

934/934 [==============================] - 0s 47us/step - loss: 0.0453 - mse: 0.0453 - val_loss: 0.0637 - val_mse: 0.0637
Epoch 60/100
934/934 [==============================] - 0s 51us/step - loss: 0.0464 - mse: 0.0464 - val_loss: 0.0607 - val_mse: 0.0607
Epoch 61/100
934/934 [==============================] - 0s 52us/step - loss: 0.0464 - mse: 0.0464 - val_loss: 0.0631 - val_mse: 0.0631
Epoch 62/100
934/934 [==============================] - 0s 51us/step - loss: 0.0455 - mse: 0.0455 - val_loss: 0.0644 - val_mse: 0.0644
Epoch 63/100
934/934 [==============================] - 0s 55us/step - loss: 0.0480 - mse: 0.0480 - val_loss: 0.0643 - val_mse: 0.0643
Epoch 64/100
934/934 [==============================] - 0s 55us/step - loss: 0.0450 - mse: 0.0450 - val_loss: 0.0596 - val_mse: 0.0596
Epoch 65/100
934/934 [==============================] - 0s 54us/step - loss: 0.0451 - mse: 0.0451 - val_loss: 0.0624 - val_mse: 0.0624
Epoch 66/100
934/934 [==============================] - 0s 53us/step

In [14]:
#評価
test_mse = model.evaluate(X_test, y_test, verbose=0)
print("test_mse : ", test_mse[0])

test_mse :  0.05575376228518682


### 【問題6】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [4]:
#MNISTデータ
from keras.datasets import mnist
(X_train_, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
#平滑化
X_train_ = X_train_.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

#正規化
X_train_ = X_train_.astype(np.float)
X_test = X_test.astype(np.float)
X_train_ /= 255
X_test /= 255

#ワンホットエンコーディング
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_train_, y_train_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import CSVLogger

#学習
model = Sequential()
model.add(Dense(input_dim=784, output_dim=50, bias=True, activation='relu'))
model.add(Dense(input_dim=50, output_dim=100, bias=True, activation='relu'))
model.add(Dense(input_dim=100, output_dim=10, bias=True, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
csv_logger = CSVLogger('log.csv', separator=',', append=False)
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=30,
                    validation_data=(X_test, y_test))

/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, activation="relu", units=50, use_bias=True)`
  if __name__ == '__main__':
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, activation="relu", units=100, use_bias=True)`
  # Remove the CWD from sys.path while we load stuff.
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, activation="softmax", units=10, use_bias=True)`
  # This is added back by InteractiveShellApp.init_path()



Train on 38400 samples, validate on 12000 samples
Epoch 1/30
38400/38400 [==============================] - 4s 94us/step - loss: 0.3432 - accuracy: 0.8996 - val_loss: 0.1921 - val_accuracy: 0.9400
Epoch 2/30
38400/38400 [==============================] - 3s 81us/step - loss: 0.1556 - accuracy: 0.9529 - val_loss: 0.1302 - val_accuracy: 0.9603
Epoch 3/30
38400/38400 [==============================] - 3s 81us/step - loss: 0.1135 - accuracy: 0.9658 - val_loss: 0.1129 - val_accuracy: 0.9654
Epoch 4/30
38400/38400 [==============================] - 3s 82us/step - loss: 0.0912 - accuracy: 0.9722 - val_loss: 0.1041 - val_accuracy: 0.9686
Epoch 5/30
38400/38400 [==============================] - 3s 81us/step - loss: 0.0731 - accuracy: 0.9769 - val_loss: 0.1042 - val_accuracy: 0.9677
Epoch 6/30
38400/38400 [==============================] - 3s 82us/step - loss: 0.0602 - accuracy: 0.9806 - val_loss: 0.1106 - val_accuracy: 0.9677
Epoch 7/30
38400/38400 [==============================] - 3s 82us/s

In [7]:
#推定
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)
pred_train = np.argmax(pred_train, axis=1) #一番大きな値のインデックスを出力
pred_test = np.argmax(pred_test, axis=1)

np.set_printoptions(threshold=0)
print(pred_train)
print(pred_train == np.argmax(y_train, axis=1))
print("---------------------")
print(pred_test)
print(pred_test == np.argmax(y_test, axis=1))

[9 0 7 ... 1 7 5]
[ True  True  True ...  True  True  True]
---------------------
[3 6 6 ... 5 1 6]
[ True  True  True ...  True  True  True]


In [8]:
#評価
train_score = model.evaluate(X_train, y_train)
test_score = model.evaluate(X_test, y_test)

12000/12000 [==============================] - 0s 20us/step


In [10]:
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])
print("----------------------------")
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

Train loss: 0.006110339705029695
Train accuracy: 0.9982812404632568
----------------------------
Test loss: 0.19634726526020313
Test accuracy: 0.9715833067893982
